In [ ]:
import boto3
import os
from dotenv import load_dotenv
from mypy_boto3_s3 import S3Client
from botocore.config import Config
import datetime
import uuid
import base64

load_dotenv(dotenv_path=os.path.join("../.env"), override=True)

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = str(os.environ.get("AWS_BUCKET_NAME"))
REGION_NAME = str(os.environ.get("AWS_REGION_NAME"))

req = {
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY_ID,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_ACCESS_KEY,
    "BUCKET_NAME": BUCKET_NAME,
    "REGION_NAME": REGION_NAME,
}

for key, value in req.items():
    if value is None:
        raise ValueError(f"{key} is not set in the environment variables.")


In [ ]:
TEST_IMAGE_PATH = 'img/test-img-sq-person-1.jpg'

s3_client: S3Client = boto3.client(
    's3',
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    endpoint_url=f"https://s3.{REGION_NAME}.amazonaws.com", # 2025-11-12 I don't know why but for ap.northeast-2, this is needed.
    # config=Config(signature_version="s3v4", s3={"addressing_style": "virtual"}),
) # type: ignore

# GEnerate object name based on timstamp + random string
import datetime
import uuid
# Generate a unique filename
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d%H%M") # Current timestamp
random_string = uuid.uuid4().hex[:12]  # 8-character random string
filename, file_extension = os.path.splitext(os.path.basename(TEST_IMAGE_PATH)) # Extract the filename and extension
OBJECT_NAME = f"{timestamp}_{random_string}{file_extension}"


try:
    with open(TEST_IMAGE_PATH, 'rb') as file:
        s3_client.upload_fileobj(file, BUCKET_NAME, OBJECT_NAME)

    # s3_client.upload_file(UPLOAD_FILE, BUCKET_NAME, OBJECT_NAME) # type: ignore
    print(f"File '{TEST_IMAGE_PATH}' uploaded to '{BUCKET_NAME}/{OBJECT_NAME}'")

    url = s3_client.generate_presigned_url(
        'get_object',
        Params={
            'Bucket': BUCKET_NAME,
            'Key': OBJECT_NAME,
            'ResponseContentType': 'image/jpeg'
        },
        ExpiresIn=3600
    )
    print(f"Presigned URL: {url}")

except Exception as e:
    print(f"Error: {e}")